In [ ]:
from image_split_script import split_yolo_img_set
from pathlib import Path

In [8]:
cwd = Path.cwd().parent
src = cwd/'data'/'labeled'/'not_split'/'labeled.zip'
dst = cwd/'data'/'labeled'
split_yolo_img_set(src, dst, test_size=0.2, random_state=42)

  20/20 items processed. (0 failed)
  6/6 items processed. (0 failed)
  20/20 items processed. (0 failed)
  6/6 items processed. (0 failed)


In [20]:
from image_split_script import load_yolo_dir
import pandas as pd

In [38]:
df = pd.DataFrame({'asda':[5,7,0,6,34,87,235,645,213,41],'besd':['1','1','321','142','123','42','24','214','42','24']})
df.sort_values('asda', axis=0, key= lambda x: x.astype(int), ignore_index=True)

,asda,besd
0,0,321
1,5,1
2,6,142
3,7,1
4,34,123
5,41,24
6,87,42
7,213,42
8,235,24
9,645,214


In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml